In [20]:
from urllib.request import urlopen
import certifi
import json
from datetime import datetime
import calendar

import pandas as pd

In [2]:
API_KEY = "e558vSI8LyRrCGv5TqWJtFTAcMDoVD7c"

FETCH_STOCKS_URL = (
    f"https://financialmodelingprep.com/api/v3/stock-screener?apikey={API_KEY}&"
    f"exchange=nyse,nasdaq&isEtf=false&isFund=false&isActivelyTrading=true&limit=10000"
)

# gross profit, earningspersharebasic
FETCH_INCOME_URL = f"https://financialmodelingprep.com/api/v3/income-statement/%s?period=quarter&limit=4&apikey={API_KEY}"
# totalAssets
FETCH_BALANCE_URL = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/%s?period=quarter&limit=4&apikey={API_KEY}"
# priceToSalesRatio
FETCH_RATIO_URL = f"https://financialmodelingprep.com/api/v3/ratios/%s?period=quarter&limit=4&apikey={API_KEY}"
# grossProfitGrowth, ebitgrowth
FETCH_GROWTH_URL = f"https://financialmodelingprep.com/api/v3/financial-growth/%s?period=quarter&limit=4&apikey={API_KEY}"

In [3]:
def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

In [4]:
stocks = get_jsonparsed_data(FETCH_STOCKS_URL)
len(stocks)

/var/folders/d9/3tp2f4s97kn5_bdphqnnqfjw0000gn/T/ipykernel_55097/654040892.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


5565

In [5]:
tickers = list(map(lambda stock: stock["symbol"], stocks))
tickers = sorted(tickers)

In [7]:
# 0 < psr < 2
# 0 < gross profit / total asset < 2
# 0 < gross_profit
# 0 < eps
# 0.1 < ebit growth
# 0.1 < gross profit growth

In [6]:
def fetch_metric(key, data):
    return data[key] if key in data else 0

def fetch_metrics(ticker):
    incomes = get_jsonparsed_data(FETCH_INCOME_URL % ticker)
    balances = get_jsonparsed_data(FETCH_BALANCE_URL % ticker)
    ratios = get_jsonparsed_data(FETCH_RATIO_URL % ticker)
    growths = get_jsonparsed_data(FETCH_GROWTH_URL % ticker)

    results = []

    size = min(len(incomes), len(balances), len(ratios), len(growths))

    for i in range(size):
        income = incomes[i]
        balance = balances[i]
        ratio = ratios[i]
        growth = growths[i]

        date = datetime.strptime(income["fillingDate"], "%Y-%m-%d")

        results.append(build_key_metrics(date, income, balance, ratio, growth))

    return results


def build_key_metrics(date, income, balance, ratio, growth):
    grossProfit = fetch_metric("grossProfit", income)
    totalAssets = fetch_metric("totalAssets", balance)

    return {
        "date": date,
        "year": date.year,
        "month": date.month,
        "period": income["period"],
        "symbol": income["symbol"],
        "revenue": fetch_metric("revenue", income),
        "gross_profit": grossProfit,
        # 매출 총 이익률 (매출 - 매출 원가) / 매출 -> 영업 효율 척도
        "gross_profit_ratio": fetch_metric("grossProfitRatio", income),
        # earnings before interest, taxes, depreciation and amortization
        # 순이익 + 감가상각비 (유 & 무형) + 세금 + 이자
        # 기업이 건강한지를 보여주는 지표
        "ebitda": fetch_metric("ebitda", income),
        # ev / ebitda
        "ebitda_ratio": fetch_metric("ebitdaratio", income),
        "operating_income": fetch_metric("operatingIncome", income),
        # 순이익
        "net_income": fetch_metric("netIncome", income),
        "net_income_ratio": fetch_metric("netIncomeRatio", income),
        "eps": fetch_metric("eps", income),
        "eps_diluted": fetch_metric("epsdiluted", income),
        "assets": totalAssets,
        "equity": fetch_metric("totalEquity", balance),
        "debt": fetch_metric("totalDebt", balance),
        "net_debt": fetch_metric("netDebt", balance),
        "gp_a": grossProfit / totalAssets if totalAssets != 0 else 0,
        # 유동 비율 : 유동 자산 / 유동 부채. 높을수록 안전
        "current_ratio": fetch_metric("currentRatio", ratio),
        # 당좌 비율 : 당좌 자산 / 유동 부채.
        "quick_ratio": fetch_metric("quickRatio", ratio),
        "cash_ratio": fetch_metric("cashRatio", ratio),
        "operating_profit_margin": fetch_metric("operatingProfitMargin", ratio),
        # return on assets. 총자산수익률
        "roa": fetch_metric("returnOnAssets", ratio),
        # return on equity
        "roe": fetch_metric("returnOnEquity", ratio),
        "net_income_per_ebt": fetch_metric("netIncomePerEBT", ratio),
        "debt_ratio": fetch_metric("debtRatio", ratio),
        "per": fetch_metric("priceEarningsRatio", ratio),
        "pbr": fetch_metric("priceToBookRatio", ratio),
        "psr": fetch_metric("priceToSalesRatio", ratio),
        "pcfr": fetch_metric("priceCashFlowRatio", ratio),
        "pegr": fetch_metric("priceEarningsToGrowthRatio", ratio),
        "revenue_growth": fetch_metric("revenueGrowth", growth),
        "gross_profit_growth": fetch_metric("grossProfitGrowth", growth),
        "ebit_growth": fetch_metric("ebitgrowth", growth),
        "operating_income_growth": fetch_metric("operatingIncomeGrowth", growth),
        "net_income_growth": fetch_metric("netIncomeGrowth", growth),
        "eps_growth": fetch_metric("epsgrowth", growth),
        "eps_diluted_growth": fetch_metric("epsdilutedGrowth", growth),
        "operating_cash_flow_growth": fetch_metric("operatingCashFlowGrowth", growth)
    }

def corps_total_rank_by_standard_columns(df, column_map) -> pd.DataFrame:
    report_df = df.copy()
    rank_df = pd.DataFrame(index=report_df.index)
    total_value = pd.Series([0] * len(report_df), index=report_df.index)

    for col, ascending in column_map.items():
        total_value += report_df[col].rank(ascending=ascending)

    rank_df['total_rank'] = total_value.rank()
    rank_df = rank_df.sort_values(by='total_rank')
    report_df['rank'] = rank_df['total_rank']

    return report_df.loc[rank_df.index]

In [7]:
metrics = []

for ticker in tickers:
    try:
        metrics.append(fetch_metrics(ticker))
        print(f"======= {ticker} ========")
    except:
        None

/var/folders/d9/3tp2f4s97kn5_bdphqnnqfjw0000gn/T/ipykernel_55097/654040892.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


======= A ========
======= AA ========
======= AACG ========
======= AACT ========
======= AADI ========
======= AAGR ========
======= AAL ========
======= AAME ========
======= AAOI ========
======= AAON ========
======= AAP ========
======= AAPL ========
======= AAT ========
======= AB ========
======= ABAT ========
======= ABBV ========
======= ABCB ========
======= ABCL ========
======= ABEO ========
======= ABEV ========
======= ABG ========
======= ABL ========
======= ABLLL ========
======= ABLV ========
======= ABM ========
======= ABNB ========
======= ABOS ========
======= ABR ========
======= ABSI ========
======= ABT ========
======= ABTS ========
======= ABUS ========
======= ABVC ========
======= ABVE ========
======= ABVX ========
======= AC ========
======= ACA ========
======= ACAD ========
======= ACB ========
======= ACCD ========
======= ACCO ========
======= ACDC ========
======= ACEL ========
======= ACET ========
======= ACGL ========
======= ACHC ========
======

======= ARKO ========
======= ARKR ========
======= ARL ========
======= ARLO ========
======= ARLP ========
======= ARM ========
======= ARMK ========
======= AROC ========
======= AROW ========
======= ARQ ========
======= ARQQ ========
======= ARQT ========
======= ARR ========
======= ARRY ========
======= ARTL ========
======= ARTNA ========
======= ARTV ========
======= ARTW ========
======= ARVN ========
======= ARW ========
======= ARWR ========
======= AS ========
======= ASA ========
======= ASAI ========
======= ASAN ========
======= ASB ========
======= ASBA ========
======= ASC ========
======= ASGN ========
======= ASH ========
======= ASIX ========
======= ASLE ========
======= ASMB ========
======= ASML ========
======= ASND ========
======= ASNS ========
======= ASO ========
======= ASPI ========
======= ASPN ========
======= ASPS ========
======= ASPU ========
======= ASR ========
======= ASRT ========
======= ASRV ========
======= ASST ========
======= ASTC ========


======= BON ========
======= BOOM ========
======= BOOT ========
======= BORR ========
======= BOSC ========
======= BOTJ ========
======= BOW ========
======= BOWL ========
======= BOWN ========
======= BOX ========
======= BOXL ========
======= BP ========
======= BPMC ========
======= BPOP ========
======= BPRN ========
======= BPTH ========
======= BPYPM ========
======= BPYPO ========
======= BQ ========
======= BR ========
======= BRAC ========
======= BRAG ========
======= BRBR ========
======= BRC ========
======= BRCC ========
======= BRDG ========
======= BREA ========
======= BRFH ========
======= BRFS ========
======= BRID ========
======= BRK-A ========
======= BRK-B ========
======= BRKH ========
======= BRKL ========
======= BRKR ========
======= BRLS ========
======= BRLT ========
======= BRNS ========
======= BRO ========
======= BROG ========
======= BROS ========
======= BRSP ========
======= BRT ========
======= BRTX ========
======= BRX ========
======= BRY =======

======= CMMB ========
======= CMND ========
======= CMP ========
======= CMPO ========
======= CMPR ========
======= CMPS ========
======= CMPX ========
======= CMRA ========
======= CMRE ========
======= CMRX ========
======= CMS ========
======= CMSA ========
======= CMSC ========
======= CMSD ========
======= CMTG ========
======= CMTL ========
======= CNA ========
======= CNC ========
======= CNDA ========
======= CNDT ========
======= CNET ========
======= CNEY ========
======= CNF ========
======= CNFR ========
======= CNGL ========
======= CNH ========
======= CNI ========
======= CNK ========
======= CNM ========
======= CNMD ========
======= CNNE ========
======= CNO ========
======= CNOB ========
======= CNP ========
======= CNQ ========
======= CNS ========
======= CNSL ========
======= CNSP ========
======= CNTA ========
======= CNTB ========
======= CNTM ========
======= CNTX ========
======= CNTY ========
======= CNVS ========
======= CNX ========
======= CNXC ========
==

...

======= PATH ========
======= PATK ========
======= PAVM ========
======= PAVS ========
======= PAX ========
======= PAY ========
======= PAYC ========
======= PAYO ========
======= PAYS ========
======= PAYX ========
======= PB ========
======= PBA ========
======= PBBK ========
======= PBF ========
======= PBFS ========
======= PBH ========
======= PBHC ========
======= PBI ========
======= PBLA ========
======= PBM ========
======= PBMWW ========
======= PBPB ========
======= PBR ========
======= PBR-A ========
======= PBYI ========
======= PCAR ========
======= PCB ========
======= PCG ========
======= PCH ========
======= PCOR ========
======= PCRX ========
======= PCSA ========
======= PCSC ========
======= PCT ========
======= PCTY ========
======= PCVX ========
======= PCYO ========
======= PD ========
======= PDCC ========
======= PDCO ========
======= PDD ========
======= PDEX ========
======= PDFS ========
======= PDLB ========
======= PDM ========
======= PDS ========
=====

======= REBN ========
======= RECT ========
======= REE ========
======= REED ========
======= REFI ========
======= REFR ========
======= REG ========
======= REGN ========
======= REKR ========
======= RELI ========
======= RELL ========
======= RELX ========
======= RELY ========
======= RENB ========
======= RENE ========
======= RENT ========
======= REPL ========
======= RERE ========
======= RES ========
======= RETO ========
======= REVB ========
======= REVG ========
======= REX ========
======= REXR ========
======= REYN ========
======= REZI ========
======= RF ========
======= RF-PF ========
======= RFAI ========
======= RFAIR ========
======= RFAIU ========
======= RFIL ========
======= RFL ========
======= RGA ========
======= RGC ========
======= RGCO ========
======= RGEN ========
======= RGF ========
======= RGLD ========
======= RGLS ========
======= RGNX ========
======= RGP ========
======= RGR ========
======= RGS ========
======= RGTI ========
======= RH ========


======= SLXNW ========
======= SM ========
======= SMAR ========
======= SMBC ========
======= SMBK ========
======= SMC ========
======= SMCI ========
======= SMFG ========
======= SMFL ========
======= SMG ========
======= SMHI ========
======= SMID ========
======= SMIT ========
======= SMLR ========
======= SMMT ========
======= SMP ========
======= SMPL ========
======= SMR ========
======= SMRT ========
======= SMSI ========
======= SMTC ========
======= SMTI ========
======= SMWB ========
======= SMX ========
======= SMXT ========
======= SN ========
======= SNA ========
======= SNAL ========
======= SNAP ========
======= SNAX ========
======= SNBR ========
======= SNCR ========
======= SNCY ========
======= SND ========
======= SNDA ========
======= SNDL ========
======= SNDR ========
======= SNDX ========
======= SNES ========
======= SNEX ========
======= SNFCA ========
======= SNGX ========
======= SNN ========
======= SNOA ========
======= SNOW ========
======= SNPS =======

======= TKNO ========
======= TKO ========
======= TKR ========
======= TLF ========
======= TLIS ========
======= TLK ========
======= TLN ========
======= TLPH ========
======= TLRY ========
======= TLS ========
======= TLSA ========
======= TLSI ========
======= TLYS ========
======= TM ========
======= TMC ========
======= TMCI ========
======= TMDX ========
======= TME ========
======= TMHC ========
======= TMO ========
======= TMTC ========
======= TMUS ========
======= TNC ========
======= TNDM ========
======= TNET ========
======= TNFA ========
======= TNGX ========
======= TNK ========
======= TNL ========
======= TNON ========
======= TNXP ========
======= TNYA ========
======= TOI ========
======= TOL ========
======= TOMZ ========
======= TOP ========
======= TOPS ========
======= TORO ========
======= TOST ========
======= TOUR ========
======= TOVX ========
======= TOWN ========
======= TOYO ========
======= TPB ========
======= TPC ========
======= TPG ========
======= 

======= VTMX ========
======= VTOL ========
======= VTR ========
======= VTRS ========
======= VTS ========
======= VTSI ========
======= VTVT ========
======= VTYX ========
======= VUZI ========
======= VVI ========
======= VVOS ========
======= VVPR ========
======= VVV ========
======= VVX ========
======= VXRT ========
======= VYGR ========
======= VYNE ========
======= VYX ========
======= VZ ========
======= VZIO ========
======= W ========
======= WAB ========
======= WABC ========
======= WAFD ========
======= WAFU ========
======= WAL ========
======= WALD ========
======= WAT ========
======= WATT ========
======= WAVE ========
======= WAVS ========
======= WAY ========
======= WB ========
======= WBA ========
======= WBD ========
======= WBS ========
======= WBTN ========
======= WBUY ========
======= WBX ========
======= WCC ========
======= WCN ========
======= WD ========
======= WDAY ========
======= WDC ========
======= WDFC ========
======= WDH ========
======= WDS ===

In [8]:
result = []

for item in metrics:
    result.extend(item)

In [9]:
stocks_df = pd.DataFrame(sum(metrics, []))
# stocks_df.to_csv('./2024_Q3.csv', index=False)

In [11]:
filtered_stocks = stocks_df[
    (stocks_df['month'].isin([7, 8, 9, 10])) & (stocks_df['year'] == 2024)
].query((
    "0 < psr and psr < 5 and "
    "0 < gp_a and gp_a < 2 and "
    "0 < pegr and pegr < 1 and "
    "0 < gross_profit and "
    "0 < eps and "
    "0.1 < ebit_growth and "
    "0.1 < gross_profit_growth"
))
sorted_stocks = corps_total_rank_by_standard_columns(filtered_stocks, {
    'psr': True,
    'gp_a': False
})

In [12]:
sorted_stocks[["date", "year", "month", "period", "symbol", "psr", "gp_a", "gross_profit", "eps", "ebit_growth", "gross_profit_growth", "pegr"]]

,date,year,month,period,symbol,psr,gp_a,gross_profit,eps,ebit_growth,gross_profit_growth,pegr
20303,2024-10-10,2024,10,Q4,VLGEA,0.784226,0.172822,1.696530e+08,1.04,0.887484,0.147444,0.133026
5973,2024-08-01,2024,8,Q2,DTIL,1.380826,0.292618,4.851600e+07,4.70,6.766898,2.002785,0.002981
15007,2024-08-14,2024,8,Q4,PFGC,0.671581,0.117919,1.602400e+09,1.08,0.784155,0.120873,0.113638
17073,2024-08-14,2024,8,Q2,SDHC,0.917316,0.137583,5.905800e+07,0.41,0.243168,0.194056,0.674969
19019,2024-07-31,2024,7,Q2,TIPT,1.096887,0.143022,4.916910e+08,0.35,9.290058,2.757238,0.252156
...,...,...,...,...,...,...,...,...,...,...,...,...
9521,2024-08-09,2024,8,Q2,IAG,4.914054,0.029426,1.468697e+08,0.16,0.607791,0.546423,0.125163
15171,2024-08-01,2024,8,Q2,PK,4.493805,0.016474,1.510000e+08,0.31,0.345238,0.237705,0.086970
3615,2024-08-09,2024,8,Q2,CBNK,4.943723,0.023581,5.750500e+07,0.59,0.179822,0.508842,0.339264
7133,2024-08-12,2024,8,Q2,FCCO,4.934924,0.013520,2.548300e+07,0.43,0.196663,7.003455,0.363768


In [13]:
RATING_URL = "https://financialmodelingprep.com/api/v3/rating/%s?apikey=e558vSI8LyRrCGv5TqWJtFTAcMDoVD7c"

In [14]:
result = []

for symbol in sorted_stocks['symbol']:
    print(f"fetch {symbol}")
    result.extend(get_jsonparsed_data(RATING_URL % symbol))

fetch VLGEA


/var/folders/d9/3tp2f4s97kn5_bdphqnnqfjw0000gn/T/ipykernel_55097/654040892.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


fetch DTIL
fetch PFGC
fetch SDHC
fetch TIPT
fetch USFD
fetch DXPE
fetch HZO
fetch MG
fetch SCS
fetch TG
fetch VEON
fetch GAP
fetch TREE
fetch NISN
fetch ASO
fetch ARC
fetch SMP
fetch VIRC
fetch CNXN
fetch PPIH
fetch TILE
fetch TWIN
fetch ANDE
fetch MLR
fetch MRX
fetch GTIM
fetch LCII
fetch MYE
fetch CVS
fetch STRT
fetch WNC
fetch EML
fetch PPC
fetch GTEC
fetch PII
fetch MUSA
fetch SPLP
fetch CENT
fetch TTI
fetch TEX
fetch DFH
fetch MBC
fetch VHI
fetch KOP
fetch PATK
fetch GIC
fetch UHG
fetch HIHO
fetch URBN
fetch SCVL
fetch MT
fetch SAJ
fetch IESC
fetch CHRW
fetch LSEA
fetch HNI
fetch PNRG
fetch BOOM
fetch CASY
fetch CCL
fetch CUK
fetch PRIM
fetch BLDR
fetch LWAY
fetch WKC
fetch SPNT
fetch CCS
fetch SNEX
fetch DAL
fetch FDP
fetch CSTM
fetch HLLY
fetch BRFS
fetch SLVM
fetch ETD
fetch DPZ
fetch KRO
fetch EPC
fetch MHK
fetch CAKE
fetch R
fetch FTDR
fetch FAF
fetch APOG
fetch FDX
fetch MPC
fetch GCMG
fetch LBRT
fetch PFIE
fetch NRG
fetch DKS
fetch TPH
fetch CALM
fetch USAP
fetch EAT
fetch 

In [15]:
rating_df = pd.DataFrame(result)
# rating_df.query("ratingScore >= 4")
rating_df

,symbol,date,rating,ratingScore,ratingRecommendation,ratingDetailsDCFScore,ratingDetailsDCFRecommendation,ratingDetailsROEScore,ratingDetailsROERecommendation,ratingDetailsROAScore,ratingDetailsROARecommendation,ratingDetailsDEScore,ratingDetailsDERecommendation,ratingDetailsPEScore,ratingDetailsPERecommendation,ratingDetailsPBScore,ratingDetailsPBRecommendation
0,VLGEA,2024-10-18,A+,4,Buy,5,Strong Buy,4,Buy,4,Buy,4,Buy,3,Neutral,4,Buy
1,DTIL,2024-10-18,B+,3,Neutral,1,Strong Sell,5,Strong Buy,5,Strong Buy,2,Sell,2,Sell,4,Buy
2,PFGC,2024-10-18,B+,3,Neutral,4,Buy,4,Buy,4,Buy,2,Sell,3,Neutral,2,Sell
3,SDHC,2024-10-18,B+,3,Neutral,1,Strong Sell,5,Strong Buy,4,Buy,5,Strong Buy,3,Neutral,1,Strong Sell
4,TIPT,2024-10-18,B+,3,Neutral,4,Buy,4,Buy,4,Buy,2,Sell,2,Sell,2,Sell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,IAG,2024-10-18,A-,4,Buy,5,Strong Buy,5,Strong Buy,5,Strong Buy,1,Strong Sell,1,Strong Sell,3,Neutral
163,PK,2024-10-18,A-,4,Buy,4,Buy,4,Buy,4,Buy,2,Sell,3,Neutral,3,Neutral
164,CBNK,2024-10-18,A-,4,Buy,5,Strong Buy,4,Buy,4,Buy,3,Neutral,3,Neutral,2,Sell
165,FCCO,2024-10-18,B+,3,Neutral,5,Strong Buy,3,Neutral,3,Neutral,2,Sell,3,Neutral,3,Neutral


In [16]:
GRADE_URL = "https://financialmodelingprep.com/api/v3/grade/%s?apikey=e558vSI8LyRrCGv5TqWJtFTAcMDoVD7c&limit=1"

In [17]:
result = []

for symbol in rating_df.query("ratingScore >= 4")['symbol']:
    print(f"fetch {symbol}")
    result.extend(get_jsonparsed_data(GRADE_URL % symbol))

fetch VLGEA


/var/folders/d9/3tp2f4s97kn5_bdphqnnqfjw0000gn/T/ipykernel_55097/654040892.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


fetch SCS
fetch TG
fetch NISN
fetch ASO
fetch ARC
fetch SMP
fetch VIRC
fetch CNXN
fetch TWIN
fetch ANDE
fetch MLR
fetch GTIM
fetch CVS
fetch STRT
fetch WNC
fetch GTEC
fetch SPLP
fetch CENT
fetch TEX
fetch DFH
fetch VHI
fetch GIC
fetch UHG
fetch URBN
fetch MT
fetch IESC
fetch PNRG
fetch BOOM
fetch CCL
fetch CUK
fetch BLDR
fetch LWAY
fetch WKC
fetch SPNT
fetch DAL
fetch FDP
fetch HLLY
fetch ETD
fetch EPC
fetch APOG
fetch MPC
fetch PFIE
fetch TPH
fetch CALM
fetch VC
fetch GRBK
fetch FLR
fetch EME
fetch NWPX
fetch NOV
fetch MTH
fetch HURN
fetch KBH
fetch IMMR
fetch DHI
fetch WTTR
fetch PHM
fetch IP
fetch MRBK
fetch TAP
fetch OPHC
fetch APA
fetch WLFC
fetch BTU
fetch FISI
fetch APTV
fetch AGX
fetch VALE
fetch OPOF
fetch EEFT
fetch CIVB
fetch FPH
fetch IMXI
fetch IAG
fetch PK
fetch CBNK


In [18]:
grade_df = pd.DataFrame(result)
grade_df

,symbol,date,gradingCompany,previousGrade,newGrade
0,SCS,2024-06-24,Benchmark,Buy,Buy
1,TG,2016-03-01,Macquarie,Neutral,Neutral
2,ASO,2024-10-03,Evercore ISI Group,Outperform,In-Line
3,ARC,2018-08-03,B. Riley Securities,Neutral,Buy
4,SMP,2024-05-23,Roth MKM,Buy,Buy
...,...,...,...,...,...
59,FPH,2020-05-22,Wells Fargo,Equal-Weight,Equal-Weight
60,IMXI,2024-08-08,BTIG,Buy,Neutral
61,IAG,2024-10-10,National Bank Financial Inc,Sector Perform,Outperform
62,PK,2024-09-26,Wolfe Research,Outperform,Peer Perform


In [19]:
grade_df.set_index('symbol')[['previousGrade', 'newGrade']].join(rating_df.set_index('symbol')[['rating', 'ratingScore']]).loc[grade_df['symbol']].head(20)

,previousGrade,newGrade,rating,ratingScore
symbol,,,,
SCS,Buy,Buy,A,4
TG,Neutral,Neutral,A,4
ASO,Outperform,In-Line,A-,4
ARC,Neutral,Buy,A-,4
SMP,Buy,Buy,A-,4
VIRC,Outperform,Outperform,A,4
CNXN,Neutral,Buy,A,4
TWIN,Outperform,Outperform,A+,4
ANDE,Overweight,Overweight,A-,4
